In [1]:
! pip install torchprofile
! pip install -U segmentation-models-pytorch Albumentations --user 

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
from model.CNNLSTM.model import CNNLSTM
from model.STCNet.src.STCNet_se_resnext.stcnet import STCNet as STCresnet
from model.STCNet.src.STCNet_mobilenetv2.stcnet import STCNet as STCmobilenet
import torch
from torchprofile import profile_macs


In [3]:
model_CNNLSTM = CNNLSTM()
input_tensor = torch.randn(1, 15, 3, 224, 224)  # Example input shape (batch_size, channels, height, width)
flops_CNNLSTM = profile_macs(model_CNNLSTM, input_tensor)

/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::unsqueeze". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::lstm". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


In [4]:
model_STCresnet = STCresnet()
input_tensor = torch.randn(1, 8, 3, 224, 224)  # Example input shape (batch_size, channels, height, width)
flops_STCresnet = profile_macs(model_STCresnet, (input_tensor,input_tensor))


    Initializing STCNet with base model: se_resnext50_32x4d.
    STCNet Configurations:
        input_modality:     RGB
        num_segments:       8
        new_length:         1
        consensus_module:   avg
        dropout_ratio:      0.5
        img_feature_dim:    256
            


In [5]:
model_STCmobilenet = STCmobilenet()
input_tensor = torch.randn(1, 8, 3, 224, 224)  # Example input shape (batch_size, channels, height, width)
flops_STCmobilenet = profile_macs(model_STCmobilenet, (input_tensor,input_tensor))


    Initializing STCNet with base model: mobilenetv2.
    STCNet Configurations:
        input_modality:     RGB
        num_segments:       8
        new_length:         1
        consensus_module:   avg
        dropout_ratio:      0.5
        img_feature_dim:    256
            


In [6]:
from super_gradients.training import models
MODEL_ARCH = 'yolo_nas_l'
yolonas_model = models.get(
    MODEL_ARCH, 
    num_classes=1, 
)
input_tensor = torch.randn(1, 3, 640, 640)
flops_yolonas = profile_macs(yolonas_model, input_tensor)

[2024-04-13 12:46:53] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /home/iec-ptit/sg_logs/console.log


[2024-04-13 12:46:53] WARNING - __init__.py - Failed to import pytorch_quantization
2024-04-13 12:46:54.259458: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 12:46:54.693761: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-13 12:46:55.371577: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this versio

In [7]:
result = f"""FLOPS:\nCNNLSTM: {flops_CNNLSTM/1e9} GFlOPs\n 
      STCmobilenet: {flops_STCmobilenet/1e9} GFlOPs\n 
      STCmobilenet: {flops_STCresnet/1e9} GFlOPs\n
      YOLOv9: 103.2 GFLOPs
      Our Framework: {flops_yolonas/1e9} GFlOPs\n"""

In [8]:
print(result)

In [9]:
import segmentation_models_pytorch as smp

/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::reshape". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::permute". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::scalarimplicit". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::arange". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::meshgrid". Sk

FLOPS:
CNNLSTM: 27.242715392 GFlOPs
 
      STCmobilenet: 4.634444289 GFlOPs
 
      STCmobilenet: 68.245332993 GFlOPs

      YOLOv9: 103.2 GFLOPs
      Our Framework: 64.892252806 GFlOPs



In [12]:
import pandas as pd

In [22]:
ENCODER = 'timm-mobilenetv3_small_minimal_100'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['sky']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation
DEVICE = 'cuda'

# create segmentation model with pretrained encoder
sgs_model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)
sgs_model.eval()
df = pd.DataFrame([],columns=['size','GFLOPs'])
for size in range(32,641,32):
    print(size)
    input_tensor = torch.randn(1, 3, size, size)
    flops_SGS = profile_macs(sgs_model, input_tensor)
    df = pd.concat([df, pd.DataFrame([{'size':size,'GFLOPs':flops_SGS/1e9}])], ignore_index=True)


32
64


/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::ceil". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::feature_dropout_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::ceil". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::feature_dropout_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


96
128


/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::ceil". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::feature_dropout_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::ceil". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::feature_dropout_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


160
192


...

/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::ceil". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::feature_dropout_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


608


/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::ceil". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::feature_dropout_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


640


/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::ceil". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(
/home/iec-ptit/anaconda3/envs/yolonas/lib/python3.9/site-packages/torchprofile/profile.py:22: UserWarning: No handlers found: "aten::feature_dropout_". Skipped.
  warnings.warn('No handlers found: "{}". Skipped.'.format(


In [ ]:
f"SGS: {flops_SGS/1e9} GFlOPs\n"

'SGS: 3.21405122 GFlOPs\n'

In [25]:
df.to_csv('../sky_ground_segmentation/experimental_result/GFLOPs.csv',index=False)